In [1]:
from __future__ import division, print_function, absolute_import

import os
import numpy as np
import tensorflow as tf
import logging
from DataPrep.data_io import DataFormatter
from nn.load_params import layer_name, convShape, getWeights
from nn.utils import getTriplets, tripletLoss
from nn.model import initNetwork

logging.basicConfig(level=logging.DEBUG, filename="logfile.log", filemode="w",
                    format="%(asctime)-15s %(levelname)-8s %(message)s")

In [2]:
def reset_graph():  # Reset the graph
    if 'sess' in globals() and sess:
        sess.close()
    tf.reset_default_graph()

## GET INCEPTION WEIGHTS

In [3]:
parentPath = "/Users/sam/All-Program/App-DataSet/Deep-Neural-Nets/Models/FaceNet-Inception"
moduleWeightDict = getWeights(parentPath)

## INITIALIZE NETWORK WITHWEIGHTS

In [4]:
# tensorDict = initNetwork(moduleWeightDict)

## LOAD THE BATCH DATA:

In [5]:
from itertools import combinations



TO DO's:

1. Remove the random weight initialiazer for the last layer, and initialize it 
   with the inception net weights.

2. implement a module to save weights as checkpoints to the disk. 

3. create a function to toggle between Random weight initializer, Inception net weight initializer 
   and using the saved checkpoint for the last Inception layer.
   
4. Add more images.

5. Create a complete workflow train the network and see

6. Store 1 image encodings for the 3-4 labels you have.

7. For a new image, pass the image throught network, get the encoding and see which is the most closest face using the encoding from the step 6.

In [ ]:
from __future__ import division, print_function, absolute_import

import os
import numpy as np
from sklearn.svm import SVC
from sklearn.externals import joblib

where_to_dump_model = '/Users/sam/All-Program/App-DataSet/DeepFaceRecognition/data_models/classification_model'

class SVM():
    '''
    # The embeddings in a nutshell are features. The face image goes through a complex network and results in
    # embeddings that captures complex features of a face. SVM's are good at classifying small datasets.
    # SVM are also robust to over fitting. The idea here is that we would wanna learn a SVM classifier using the
    # embeddings as the feature space and see for the given embedding, how many times we are able to predict the
    # correct class
    '''
    
    def __init__(self):
        pass
    
    def train(self, embeddings, labels, iter_num=None):
        '''
        :param embeddings:   Embeddings of the image
        :param labels:       labels
        :return:
        '''
        model = SVC(kernel='linear', probability=True)
        model.fit(embeddings, labels)
        joblib.dump(model, os.path.join(where_to_dump_model, iter_num+"_svm.sav"))
        
    def classify(self, embeddings, iter_num=None):
        '''
        :param embeddings: Image embeddings to classify
        :param iter_num:
        :return:
        '''
        model = joblib.load(os.path.join(where_to_dump_model, iter_num+"_svm.sav"))
        predLabels = model.predict_proba(embeddings)
        best_label_idx = np.argmax(predLabels, axis=1)
        print (predLabels)
        print (best_label_idx)

In [8]:
path_to_batch_data = '/Users/sam/All-Program/App-DataSet/DeepFaceRecognition/data_models/batch_img_arr'
batch_file_name = 'random_batches.pickle'
checkpoint_path = '/Users/sam/All-Program/App-DataSet/DeepFaceRecognition/data_models/saver_checkpoints'

dataX, dataY, labelDict = DataFormatter.getPickleFile(folderPath=path_to_batch_data, 
                                                      picklefileName=batch_file_name, getStats=True)

'''
dataX = [num_batches, image_per_batch, image_x, image_y, image_channels]
dataY = [num_batches, labels]

'''
print (dataX.shape, dataY.shape)
reset_graph()
encodingDict = initNetwork(moduleWeightDict,init_wght_type='random', isTrainable=True)
# add ops to save and restore model
saver = tf.train.Saver()
print (encodingDict)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    if 'model.ckpt' in os.listdir(checkpoint_path):
        saver.restore(sess, os.path.join(checkpoint_path, "model.ckpt"))
    for batchNum, batchX in enumerate(dataX):
        batchY = dataY[batchNum, :]
        opt, batch_loss = sess.run([encodingDict['optimizer'], 
                                   encodingDict['loss']], 
                                  feed_dict={encodingDict['inpTensor']:batchX})
#         print (opt)
        print('@@@@')
        print (batch_loss)
        print ('#########################')
    save_path = saver.save(sess, os.path.join(checkpoint_path, "model.ckpt"))
#         embeddings = embeddings[0]
        
#         a
#         tripletIndexArr = np.array(tripletIndexArr).reshape(-1,3)
#         a_idxs = tripletIndexArr[:,0].flatten()
#         p_idxs = tripletIndexArr[:,1].flatten()
#         n_idxs = tripletIndexArr[:,2].flatten()
#         loss = tripletLoss(tf.gather(tf.cast(embeddings, dtype=tf.float32), a_idxs),
#                            tf.gather(tf.cast(embeddings, dtype=tf.float32), p_idxs),
#                            tf.gather(tf.cast(embeddings, dtype=tf.float32), n_idxs), alpha=0.2)
#         print (loss.eval())
        
        
#         pritn ()

The shape of input data (X) is:  (10, 18, 96, 96, 3)
The shape of input data (Y) is:  (10, 18)
Unique labels in dataY is:  [ 0.  1.  2.]
Label dict:  None
(10, 18, 96, 96, 3) (10, 18)


/Users/sam/App-Setup/anaconda/envs/anaconda35/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


{'inpTensor': <tf.Tensor 'Placeholder:0' shape=(?, 96, 96, 3) dtype=float32>, 'optimizer': <tf.Operation 'Adam' type=NoOp>, 'output': <tf.Tensor 'InceptionFC_FT/L2_norm:0' shape=(?, 128) dtype=float32>, 'loss': <tf.Tensor 'TripletLoss/Sum_2:0' shape=<unknown> dtype=float32>}
@@@@
8.03572
#########################
@@@@
9.3952
#########################
@@@@
9.4741
#########################
@@@@
9.36212
#########################
@@@@
8.04453
#########################
@@@@
6.8662
#########################
@@@@
6.5465
#########################
@@@@
7.41043
#########################
@@@@
6.62778
#########################
@@@@
8.01394
#########################


In [ ]:
import tensorflow as tf
import numpy as np
np.random.seed(1)
tfdata = tf.cast(np.random.rand(1,1,3,5) + 10, dtype=tf.float32)
print (tfdata.get_shape().as_list())
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    batchMean, batchVar = tf.nn.moments(tfdata, axes=[0,1,2], name="moments")
    print (tfdata.eval())
    print (batchMean.eval())
    print (batchMean.get_shape().as_list())
    print (batchVar.eval())
    print (batchVar.get_shape().as_list())

In [44]:
np.array([1,2,3,4])[-1]

4

In [12]:
t = tf.constant([[[1, 1, 1], [2, 2, 2]],
                 [[3, 3, 3], [4, 4, 4]],
                 [[5, 5, 5], [6, 6, 6]]])

a = tf.constant([[1,2,3],[3,2,4],[1,1,4]])
print(t.shape)
print (a.shape)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print (tf.gather(a, [0, 1]).eval())



(3, 2, 3)
(3, 3)
[[1 2 3]
 [3 2 4]]


In [13]:
pow(0.3426 - 0.927, 2) + pow(0.7484-0.62853, 2)

0.3558921769

In [20]:
reset_graph()


# Create some variables.
pathsave = '/Users/sam/All-Program/App-DataSet/DeepFaceRecognition/data_models/rough'

v1 = tf.get_variable("v1", shape=[3], initializer = tf.zeros_initializer)
v2 = tf.get_variable("v2", shape=[5], initializer = tf.zeros_initializer)

inc_v1 = v1.assign(v1+1)
dec_v2 = v2.assign(v2-1)

# Add an op to initialize the variables.
init_op = tf.global_variables_initializer()

# Add ops to save and restore all the variables.
saver = tf.train.Saver()

# Later, launch the model, initialize the variables, do some work, and save the
# variables to disk.
with tf.Session() as sess:
    sess.run(init_op)
    # Do some work with the model.
    inc_v1.op.run()
    dec_v2.op.run()
    # Save the variables to disk.
    save_path = saver.save(sess, pathsave+"/model.ckpt")
    print("Model saved in file: %s" % save_path)

Model saved in file: /Users/sam/All-Program/App-DataSet/DeepFaceRecognition/data_models/rough/model.ckpt


In [21]:
tf.reset_default_graph()
# Create some variables.
v1 = tf.get_variable("v1", shape=[3])
v2 = tf.get_variable("v2", shape=[5])

# Add ops to save and restore all the variables.
saver = tf.train.Saver()

# Later, launch the model, use the saver to restore variables from disk, and
# do some work with the model.
with tf.Session() as sess:
    # Restore variables from disk.
    saver.restore(sess, pathsave+"/model.ckpt")
    print("Model restored.")
    # Check the values of the variables
    print("v1 : %s" % v1.eval())
    print("v2 : %s" % v2.eval())

INFO:tensorflow:Restoring parameters from /Users/sam/All-Program/App-DataSet/DeepFaceRecognition/data_models/rough/model.ckpt
Model restored.
v1 : [ 1.  1.  1.]
v2 : [-1. -1. -1. -1. -1.]
